In [93]:
%%time
# Program streamlines orders

# STATUS: Incomplete

import pandas as pd
import numpy as np

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
horizon = 50

# Read the main file
df_main = pd.read_pickle(datapath+'df_main.pkl')

# Compute price variance percentage
df_puts['price_var'] = (df_puts.pBidPrice - df_puts.pLTP)
df_puts['price_var_pct'] = (df_puts.pBidPrice - df_puts.pLTP)/df_puts.pBidPrice

# Establish target price and weed substitute NaN with bidprice
df_puts['tgt_price'] = np.where(df_puts.price_var < -10, df_puts.pBidPrice+0.15, df_puts.pLTP)
df_puts.loc[df_puts.tgt_price.isna(), 'tgt_price'] = df_puts.pBidPrice + 0.15

# Determine return on margin
df_puts['RoM'] = df_puts.tgt_price/df_puts.marginpct*df_puts.undPrice*252/df_puts.DTE*100

# Filter the puts with some RoM
df_puts = df_puts.loc[(df_puts.Type == 'p') & 
                      (df_puts.RoM > 0), :].sort_values(['DTE', 'RoM'], ascending=False)

cols = ['ibSymbol', 'undPrice', 'Strike', 'Expiry', 'DTE', 
        'pLTP', 'pBidPrice', 'tgt_price', 'Type', 'Mlot', 'Delta', 'RoM']

# Sort for Delta
df_puts = df_puts.sort_values(['DTE', 'Delta'], ascending=False)

# Screen timeframe within horizon
df_puts1 = df_puts[(df_puts.DTE.between(11, horizon, inclusive=False))]

# Keep only high delta records
df_puts2 = df_puts1[df_puts1.Delta >= 1]

# Pickle the puts
df_puts2.to_pickle(datapath+'df_preorder_puts.pkl')

# make csv for lower-limit of options
df_th = pd.DataFrame(df_puts2[cols].sort_values(['RoM'], 
                    ascending=False).ibSymbol.unique())
df_th['Des'] = 'DES'
df_th['Type'] = 'STK'
df_th['Market'] = 'NSE'
df_th = df_th[['Des', 0, 'Type', 'Market']].rename(columns={0: 'ibSymbol'})

# Get the min of ohlc for these symbols
df_ohlc = pd.read_pickle(datapath+'df_ohlc.pkl')

df_min = pd.DataFrame(df_ohlc.groupby('ibSymbol').C.min()).reset_index().rename(columns={'C': 'minC'})
df_th = df_th.merge(df_min)

df_th.to_csv(datapath+'put_threshold.csv', index=False, header=0)

Wall time: 105 ms


In [92]:
df_th.head()

,Des,ibSymbol,Type,Market,minC
0,DES,EICHERMOT,STK,NSE,21026.15
1,DES,APOLLOHOS,STK,NSE,927.10
2,DES,LT,STK,NSE,1117.60
3,DES,ASIANPAIN,STK,NSE,1101.60
4,DES,KOTAKBANK,STK,NSE,948.35
